# Network Rule Persistence Workaround

This notebook helps you set up a workaround to ensure that your network ingress rules persist even when they are cleared by administrative tasks.

## Problem
- Administrative task `PARENT_PROC_TASK` in `SECURITY_NETWORK_DB.POLICIES` periodically clears network rules
- This is designed to prevent internal staff forgetting to clear up their rules after a demo or working session
- However this breaks connectivity for applications that depend on specific ingress rules, like Openflow
- We need to automatically restore our required network rules immediately after they are cleared so Demos are not interrupted

## Solution
- Create a network rule for our required ingress
- Create a procedure that checks and restores the network rule
- Create a task that runs immediately after the administrative task

## Prerequisites
- `CREATE NETWORK RULE` privilege
- `CREATE PROCEDURE` privilege
- `CREATE TASK` privilege
- Access to modify network policies
- Or just ACCOUNTADMIN


In [1]:
# Configuration - Update these values for your environment
# ==========================================================

# Context where objects are created
DATABASE = "OPENFLOW";
SCHEMA = "OPENFLOW";

# Network Rule Configuration
NETWORK_RULE_NAME = "allow_openflow_deployment_ingress"
IP_ADDRESS_LIST = "('10.16.0.0/12')"  # Update with your IP addresses
NETWORK_RULE_COMMENT = "Allow ingress from Openflow deployment server - auto-restored by NETWORK_RULE_RESTORATION_TASK"

# Task Configuration
WAREHOUSE_NAME = "COMPUTE_WH"  # Update to your warehouse name
TASK_NAME = "NETWORK_RULE_RESTORATION_TASK"
PROCEDURE_NAME = "CHECK_AND_SET_NETWORK_RULE_LIST"

# Network Policy Configuration
NETWORK_POLICY_NAME = "ACCOUNT_VPN_POLICY_SE"  # Update to your network policy name

# Administrative Task Configuration
ADMIN_TASK_PATH = "SECURITY_NETWORK_DB.POLICIES.PARENT_PROC_TASK"  # Update to your admin task path

print("✓ Configuration variables set successfully!")
print(f"Network Rule: {NETWORK_RULE_NAME}")
print(f"IP Addresses: {IP_ADDRESS_LIST}")
print(f"Warehouse: {WAREHOUSE_NAME}")
print(f"Network Policy: {NETWORK_POLICY_NAME}")
print(f"Admin Task: {ADMIN_TASK_PATH}")


✓ Configuration variables set successfully!
Network Rule: allow_openflow_deployment_ingress
IP Addresses: ('10.16.0.0/12', '3.10.2.57/32')
Warehouse: COMPUTE_WH
Network Policy: ACCOUNT_VPN_POLICY_SE
Admin Task: SECURITY_NETWORK_DB.POLICIES.PARENT_PROC_TASK


## Step 0: Set Context

In [2]:
USE DATABASE {{DATABASE}};
USE SCHEMA {{SCHEMA}};

SyntaxError: invalid syntax (3329054906.py, line 1)

## Step 1: Create the Network Rule

Create the network rule using the configuration variables defined above.


In [ ]:
-- Create network rule for Openflow deployment ingress
CREATE OR REPLACE NETWORK RULE {{NETWORK_RULE_NAME}}
  MODE = INGRESS
  TYPE = IPV4
  VALUE_LIST = {{IP_ADDRESS_LIST}}
  COMMENT = '{{NETWORK_RULE_COMMENT}}';


In [ ]:
-- Verify the network rule was created
DESCRIBE NETWORK RULE {{NETWORK_RULE_NAME}};

### Step 1.1 Check your Network Policy
Describe your Network Policy, and see if you already have rules set

In [ ]:
DESCRIBE NETWORK POLICY {{NETWORK_POLICY_NAME}};

### Step 1.2 Set the Rule
If you have previously used a rule with different IPs, and want to set a new rule now, run this cell

In [ ]:
ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}} ADD ALLOWED_NETWORK_RULE_LIST = ({{NETWORK_RULE_NAME}});

## Step 2: Create the Network Rule Restoration Procedure

This procedure checks if the network rule is present in the network policy and restores it if missing.


In [ ]:
CREATE OR REPLACE PROCEDURE {{PROCEDURE_NAME}}()
RETURNS VARCHAR
LANGUAGE SQL
EXECUTE AS CALLER
AS '
DECLARE
    rule_list_count INTEGER DEFAULT 0;
BEGIN
    -- Describe the network policy to get current settings
    DESCRIBE NETWORK POLICY {{NETWORK_POLICY_NAME}};

    -- Check if ALLOWED_NETWORK_RULE_LIST is present
    SELECT COUNT(*)
    INTO :rule_list_count
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
    WHERE "name" = ''ALLOWED_NETWORK_RULE_LIST''
    AND "value" IS NOT NULL
    AND "value" != '''';

    -- If ALLOWED_NETWORK_RULE_LIST is not set, add it
    IF (:rule_list_count = 0) THEN
        ALTER NETWORK POLICY {{NETWORK_POLICY_NAME}}
        ADD ALLOWED_NETWORK_RULE_LIST = ({{NETWORK_RULE_NAME}});

        RETURN ''({{NETWORK_RULE_NAME}}) has been added to ALLOWED_NETWORK_RULE_LIST'';
    ELSE
        RETURN ''ALLOWED_NETWORK_RULE_LIST is already configured'';
    END IF;
END;
';

In [ ]:
-- Test the procedure
CALL {{PROCEDURE_NAME}}();

## Step 3: Create the Network Rule Restoration Task

Since the parent task runs every 12 hours (at midnight and noon UTC), we'll schedule our task to run 2 minutes after those times. This ensures we restore the network rules shortly after they're cleared, while keeping our task in the OPENFLOW.PUBLIC schema.


In [ ]:
-- Inspect the parent task to see its current schedule
-- This helps verify when it runs so we can schedule our task appropriately
DESCRIBE TASK {{ADMIN_TASK_PATH}};

-- Get the Cron schedule
SELECT "schedule"
FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));

In [ ]:
-- Create task that runs 2 minutes after the parent task (probably every 12 hours at 00:02 and 12:02 UTC)
CREATE OR REPLACE TASK {{TASK_NAME}}
    WAREHOUSE = {{WAREHOUSE_NAME}}
    SCHEDULE = 'USING CRON 2 */12 * * * UTC'
    COMMENT = 'Restore ALLOWED_NETWORK_RULE_LIST 2 minutes after admin task clears rules (every 12 hours)'
    AS CALL {{PROCEDURE_NAME}}();

## Step 4: Start the Task

Enable the task to begin monitoring and restoring network rules.


In [ ]:
-- Start the task
ALTER TASK {{TASK_NAME}} RESUME;

In [ ]:
-- Verify the task is running
SHOW TASKS LIKE '{{TASK_NAME}}';

## Step 5: Verification and Monitoring

Use these queries to monitor the task execution and verify the network rules are being maintained.


In [ ]:
-- Check current network policy settings
DESCRIBE NETWORK POLICY {{NETWORK_POLICY_NAME}};

In [ ]:
-- Check task execution history. It should be SCHEDULED if you've just set it up.
SELECT
    name,
    state,
    scheduled_time,
    completed_time,
    return_value,
    error_code,
    error_message
FROM table(information_schema.task_history())
WHERE name = '{{TASK_NAME}}'
ORDER BY scheduled_time DESC
LIMIT 10;

In [ ]:
-- Show all Tasks
-- SHOW TASKS;

-- Pause the task if needed
-- ALTER TASK {{TASK_NAME}} SUSPEND;

-- Check all network rules
-- SHOW NETWORK RULES;

-- Check all network policies
-- SHOW NETWORK POLICIES;

-- Get task definition
-- SELECT GET_DDL('TASK', '{{TASK_NAME}}');
